## Crispy CRISPRs
David Tian, Audra Devoto, Zoey Werbin and Justine Albers

### Project Overview

We studied bacterial CRISPR-Cas systems in the Sippewissett Salt Marsh pink berry consortium. We first constructed a phylogenetic tree for the pink berry bacteria. We then matched the CRISPR sequences from full unassembled metagenomic reads of *Bacteroidetes*, *Alphaproteobacteria*, purple sulfur bacteria, and sulfate reducing bacteria found in the pink berries with sequences of potential target phage. We created a database of non-CRISPR reads from the metagenome to search for SNC (spacer-containing non-CRISPR) reads. In addition, we BLAST-searched the SNCs against the full NCBI database filtered for sequences that contain ‘phage’ in the title to determine if they matched phage sequences.

###Making phylogenetic tree

First, we wanted to **determine how all of the microbial species from the pink berry metagenomic data are related**. In order to do this, we created a phylogenetic tree: 

 - Obtain *recA* sequences for PSB, SRB, the three *Alphaproteobacteria* species, and the five *Bacteroidetes* species from MG-RAST (*recA* is a housekeeping gene used for DNA repair, found across all species).
 - Get some reference *recA* sequences from NCBI from other related bacteria.

 - Navigate to the FASTA files of these recA sequences and copy and paste them into Geneious as sequences. 

 - Select all of the sequences and click on ‘Align/Assemble’ to use the ‘Multiple Alignment’ tool to align all of the sequences. 
 - Select the resulting multiple alignment file and click on the ‘Tree’ tool to build the tree. 

 - There are several options / functions that you can select from for the design of your tree. For the purposes of just creating a phylogenetic tree, you can leave many of the defaults alone. 

 - You can select:
    -  A genetic distance model
    -  A tree building method
    -  An outgroup (optional)

 - Make sure to check the box for ‘Resample Tree’ to include some bootstrap replicates. Choose the option to create a consensus tree and set the support threshold % to zero. 
 
### Extracting CRISPR spacers

We primarily used a program called CRASS in order to identify CRISPR spacers in the pink berry metagenome. Download the following programs: 
 
Apache (https://httpd.apache.org/download.cgi) - a collaborative software development effort aimed at creating a robust, commercial-grade, featureful, and freely-available source code implementation of an HTTP (Web) server

Zlib (http://zlib.net/) - software library used for data compression

Bedtools (https://github.com/arq5x/bedtools2/releases) - a swiss-army knife of tools for a wide-range of genomics analysis tasks. The most widely-used tools enable genome arithmetic: that is, set theory on the genome. For example, bedtools allows one to intersect, merge, count, complement, and shuffle genomic intervals from multiple files in widely-used genomic file formats such as BAM, BED, GFF/GTF, VCF.

CRASS (http://ctskennerton.github.io/crass/) - CRASS is a tool for finding and assembling reads from genomic and metagenomic datasets that contain Clustered Regularly Interspersed Short Palindromic Repeats (CRISPR). CRASS searches through the dataset and identifies reads which contain repeated K-mers that are of a specific length and are separated by a spacer sequence. These possible direct repeats are then curated internally to remove bad matches and then reads containing direct repeats are then outputed for further analysis.

**Use CRASS to extract CRISPR arrays from the assembled metagenome:**

**Input:** whole-metagenome-assemblies/moleculo-assembly/asmMeta.all.fasta (Moleculo metagenome assembled into contigs)

**Output**: crass.crispr file (for information on the .crispr format, see http://ctskennerton.github.io/crass/assets/crispr_spec.pdf), fasta files containing the contigs that each array was found on (each array is labeled as a group, i.e. G1, G2, G3, etc.), graphs (.gv files) for each CRISPR array linking the spacers in their putative order.

In [ ]:
crass asmMeta.all.fasta

###Removing CRISPR-containing reads from metagenome to create BLAST database

Creating a new BLAST database that has all of the CRISPR-containing reads removed will allow us to search for sequences in the metagenome that match the CRISPRs. Our first step is to remove all the reads from the metagenome that contain CRISPR reads. Use the following command to **combine all the contigs that were found to have CRISPR arrays on them:**

In [ ]:
cat Group_*.fa > all_crispr_reads.fa

**Install BioPython**, a Python package for bioinformatics:

In [ ]:
sudo pip install biopython

Create the following script to parse through the metagenome and **remove all contigs containing arrays** (present in all_crispr_reads.fa):

Name: fasta_remove.py

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys
from Bio import SeqIO

fasta_file = sys.argv[1]  # Input fasta file
remove_file = sys.argv[2] # Input wanted file, one gene name per line
result_file = sys.argv[3] # Output fasta file

remove = set()
with open(remove_file) as f:
    for line in f:
        line = line.strip()
        if line != "":
            remove.add(line)

fasta_sequences = SeqIO.parse(open(fasta_file),'fasta')

with open(result_file, "w") as f:
    for seq in fasta_sequences:
        nuc = seq.seq.tostring()
        if nuc not in remove and len(nuc) > 0:
            SeqIO.write([seq], f, "fasta")


**Call the script.** Arguments are positional, sys.argv[1] = input file, sys.argv[2] = fasta with sequences you wish to remove from the input file, sys.argv[3] = name of output file.


In [ ]:
python fasta_remove.py asmMeta.fasta all_crispr_reads.fa no_crispr_asmMeta.fasta

Unfortunately the output file is not a linear fasta file. Each line is a set number of characters, which you can see using the head command.

In [ ]:
head no_crispr_asmMeta.fasta

**Linearize this file:**

In [ ]:
awk '/^>/ {printf("%s%s\t",(N>0?"\n":""),$0);N++;next;} {printf("%s",$0);} END {printf("\n");}' < no_crispr_asmMeta.fasta > no_crispr_asmMeta.linear.fasta.temp

tr "\t" "\n" < no_crispr_asmMeta.linear.fasta.temp > no_crispr_asmMeta.linear.fasta

(You may delete the .temp file once you have confirmed that the process worked.)

**Check** that the output file does not contain any reads with CRISPR arrays. This should return “no CRISPR arrays found”.

In [ ]:
crass no_crispr_asmMeta.linear.fasta

**Create new BLAST database** from the genome with all CRISPR-containing reads removed.

In [ ]:
makeblastdb -in no_crispr_asmMeta.linear.fasta -parse_seqids -dbtype nucl -out asmMetaNoCRISPR

**Generate** a fasta file of all the spacer sequences from all the CRISPR arrays found.

In [ ]:
crisprtools extract -s crass.crispr > all_spacers.fasta

**Search** that fasta file against the asmMetaNoCRISPR database you just created using blastn.

In [ ]:
blastn -db asmMetaNoCRISPR/asmMetaNoCRISPR -query all_spacers.fasta -out spacers_metagenome.classified.csv

**Separate** out the contigs from asmMetaNoCRISPR that had a match to a spacer (these will be referred to as **SNCs: spacer-containing non crispr contigs**):

Create a python script to do this:
Name: parse_spacer_classifications.py

In [ ]:
with open("spacers_metagenome.classified.csv", "r") as f:
	with open("snc_ids.txt", "a") as snc_ids:
		with open("snc_matches.tsv", "a") as out:
			for line in f:
				if line.split("= ")[0]=="Query":
					spacer = line.split("= ")[1].rstrip("\n")
				if line.startswith("*"):
					value = "NA"
					#print "Spacer", spacer, "Value ", value
					out.write(spacer+"\t"+value+"\n")
				elif line.startswith(">"):
					value = line.rstrip('\n').rstrip(' ')
					out.write(spacer+"\t"+value+"\n")
					snc_ids.write(value+"\n")

**This will create two output files:**

snc_ids.txt = a text file of SNC ids

snc_matches.tsv = a .tsv file of each spacer, and the SNC id that it matched. 

If the spacer had no matches in the metagenome, it says NA. This file was later used to make Supplementary Table 1. 

However, there are a bunch of duplicates in this file. **Remove them using sort and uniq:** 

In [ ]:
sort snc_ids.txt | uniq > unique_snc_ids.txt

**Find these SNC reads in the metagenome** based on their ID:

In [ ]:
seqtk no_crispr_asmMeta.linear.fasta snc_ids.txt > snc_reads.fasta

**Great!** Now we have a fasta file of all the SNC reads. 

###Identifying the origin of the SNC reads using blastx

*Note: this never actually worked for us! We were hoping to find out what percentage of the SNC reads had blastx hits to viral proteins, compared to a random sampling of reads from the metagenome. This was modeled off of Fig. S2 from Andersson and Banfield, 2008.*

**Launch an AWS EC2 instance.**
Select "AWS Marketplace". 
Search NCBI and select the NCBI blast community AMI.
Launch the instance, and copy snc_reads.fasta over to the instance.

From the instance terminal, **run blastx**:

In [ ]:
blastx -query snc_reads.fasta -out 'SNC_classifications_blastx.csv' -db nr -outfmt '10 qseqid sseqid sacc slen qstart qend sstart send qseq sseq evalue bitscore length pident qcovs staxid ssciname scomname sblastname sskingdom stitle' -max_target_seqs 5 -num_threads [AS MANY AS YOU HAVE]

This takes a very long time. It never finished for us, and we never got to the point of subsampling the metagenome and testing those. If we were to subsample, we would use the qiime script:

In [ ]:
subsample_fasta.py

[more info](http://qiime.org/scripts/subsample_fasta.html) on said qiime script

###Identifying spacer targets using NCBI

To find phage targets, spacer sequences were searched against the entire NCBI database (filtered by entries that had “phage” in the title) using blastn optimized for short sequences. 

This is all done from a local terminal. 

**Download** the search strategy specifying blastn search only through sequences that have “phage” in the title, and also optimized for short reads (the spacer regions are very short):

In [ ]:
wget https://raw.githubusercontent.com/oddaud/micdiv2017/master/phage_title.asn

**Run** blastn:

In [ ]:
blastn -query all_spacers.fasta -out 'Spacer_phage_blastn.classified.csv' -outfmt '10 qseqid sseqid evalue bitscore length pident  qcovs staxid ssciname scomname sblastname sskingdom stitle' -remote -import_search_strategy phage_title.asn

###Final data processing steps

**Format output file to import into R**

Open the Spacer_phage_blastn.classified.csv file in excel. Most of this processing was done by hand, which is a huge pain in the ass.

 - Add headers. The headers are ‘qseqid sseqid evalue bitscore length pident  qcovs staxid ssciname scomname sblastname sskingdom stitle’
 - Sort the file by the percent identity column. Delete everything under 100
 - Sort the file by the coverage column. Delete everything under 80.
 - Delete the extra columns of the title. This happens because the sequence titles have commas in them, so neither R nor excel knows how to deal with this. Make sure you retain the important title information, such as the fact that it was a phage and its name, but also that the stitle only takes up one column. 
 - Sort the file by qseqid, and then coverage, so that highest coverage is at the top, and qseqids are grouped together. 
 - Once this is complete, save it as a csv called ‘spacer.classified.csv’
 - Delete the headers
 - Open the file in R using the command:

In [ ]:
classified <- read.csv("path/to/spacer.classified.csv", header = FALSE, sep =",")

 - Rewrite the file using R (do not know why this is necessary but it is or you run into problems):

In [ ]:
write.csv(classified, "path/to/spacer.classified.CLEAN.csv")

**Create** a python script to parse through this file and select the top (arbitrary) hit: 

name: parse_classifications.py

In [ ]:
out = open('phage_spacer_top_hit.csv', 'a')
id = ""
with open('spacer.classified.CLEAN.csv', 'r') as f:
	for line in f:
		#print line
		if id=="":
			id = line.split(",")[0]
			out.write(line)
		if line.split(",")[1] not in id:
			out.write(line)
			id = line.split(",")[1]

		else:
			pass

**Run this script!**

In [ ]:
python parse_classifications.py

**BUT WAIT! There is STILL more data processing...**

Making Supplemental Table 1 required a lot of manual processing. 

**Find the host for each of the arrays:**

To do this, map the contig(s) that each array was found on to each of the binned genomes (berry-metagenome/genome_assemblies/*)

*This was done using Geneious:* 

 - Import all array contig fasta files (they are labelled as, for example, Group_1_AGAAACACCCCCACGGGCGTGGGGAAGAC.fa)
 - Import the metagenome assemblies. 
 - Select the first array (Group_1_)
 - Click “Align/Assemble” then “Map to Reference”. Choose all 13 binned genomes as the reference. 
 - Select “Do not trim”, then “OK”
 - If there is a hit to any of the genomes, mark it for all the spacers in that array in the excel spreadsheet. If there is not, mark the host as the name of the contig that the array was found on (open the Group_X fasta file to find this). 

**Sort by groups and spacers:**

 - Add two columns, one called “group”, one called “spacer”. 
 - Manually go through and fill these in for each spacer. 
 - ex: For G1SP27_Cov_2, the Group is 1 and the spacer is 27.
 - Save as a csv, called “spacer_classifications.csv”

Now the file should look like this: 

<img src="https://raw.githubusercontent.com/oddaud/micdiv2017/master/table_example.png"/>

**Add in the “unknown” classifications.**
Because blastn only returns classified spacers, we are missing a lot of data!

**Create a python script** to run through the file all_spacers.fasta and check that all spacers are represented in the classification file:

name: add_missing.py

In [ ]:
out = open('missing.txt', 'a')
with open('spacer_classifications.csv', r) as f:
	with open('all_spacers.fasta', r) as s:
		spacer_ids=[]
		for line in f:
			spacer_id = line.split(",")[1]
			spacer_ids.append(spacer_id)
		for line in s:
			if line.startswith(">"):
				id = line[1:]
				print id
				if id not in spacer_ids:
					out.write(id+"\n")
					#print id + "Not in classified!!"

This will output a txt file of all the spacers that are missing from the classified file. Copy these over in excel, and fill in the columns that you can for each one.

*OPTIONAL:*

Add in information about whether or not each spacer had a hit in the metagenome! Use the file snc_matches.tsv to do this. 

Save as a csv. Make sure the headers are: 
id, group, spacer, accession, evalue, unknown, unknown1, pident, taxid, match, cov, host

### Creating the network###
*(file also attached as generate_networks.R)*

In R: 

In [ ]:
library(ggplot2)
library(sharpshootR)
library(RColorBrewer)
library('Matrix')
library(bipartite)
library(igraph)

#Import Data
dataset <- read.table("[File path to the final version of the classified csv] ", header = TRUE, sep = ',', fill = TRUE)
df <- dataset

#Generate a sparse matrix based on the table. Host and match are the two modes of this matrix. 
A <- spMatrix(nrow=length(unique(df$host)),
              ncol=length(unique(df$match)),
              i = as.numeric(factor(df$host)),
              j = as.numeric(factor(df$match)),
              x = rep(1, length(as.numeric(df$host))) )

row.names(A) <- levels(factor(df$host))
colnames(A) <- levels(factor(df$match))

#Turn it into a data frame
D <- as.data.frame(as.matrix(A))

#Create a graph
G <- graph_from_incidence_matrix(A)
#Check if it is a bipartite graph!
is_bipartite(G)

###GENERATE PLOT###
##Black represents each of the unassigned contigs. Colors represent the genomes.
mycol <- c("yellow", "#1F78B4", "black","black","black","black","black","black",
           "black","black","black","black","black","black","black","black","black",
           "black","black","black","black","black","black","red", "#33A02C")

#Delete the unknown vertice, this messes up the whole plot 
#because it looks like a phage called "unknown" is attacking all the 
#genomes simultaneously. In future analysis, these unknown sequences could be included?
G <- delete_vertices(G, "unknown")

##Formatting
V(G)$color <- rep('grey', length(V(G)$name))
V(G)$color[V(G)$name !=''] <- mycol[V(G)$name !='']

plot.igraph(G, vertex.size=2, vertex.label.cex=0.1, vertex.label =NA, edge.width = 0.2)
vertex.label = V(G)$name

legend(1,1,fill = V(G)$color, legend=V(G)$name[V(G)$name !=''], cex = 0.5, xjust=3.7, yjust = 0.9)

**DONE!!**

#### Literature Cited: 

1) Andersson, A. F., & Banfield, J. F. (2008). Virus population dynamics and acquired virus resistance in natural microbial communities. Science, 320(5879), 1047-1050.